<a href="https://colab.research.google.com/github/kadimkareem/akhbaruna-privacy/blob/main/analysis/load_AIDev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AIDev dataset

In [24]:
import pandas as pd
all_pr_df = pd.read_parquet("hf://datasets/hao-li/AIDev/all_pull_request.parquet")
all_repo_df = pd.read_parquet("hf://datasets/hao-li/AIDev/all_repository.parquet")
all_user_df = pd.read_parquet("hf://datasets/hao-li/AIDev/all_user.parquet")

,count
state,
closed,859927
open,72864


In [34]:
# Merge pr_df and pr_task_type_df to analyze agent usage by task type
merged_pr_task_df = pd.merge(pr_df, pr_task_type_df, on='id', suffixes=('_pr', '_task'))

# Group by agent and task type and count the occurrences
agent_task_counts = merged_pr_task_df.groupby(['agent_pr', 'type']).size().unstack(fill_value=0)

# Display the results
display(agent_task_counts)

type,build,chore,ci,docs,feat,fix,other,perf,refactor,revert,style,test
agent_pr,,,,,,,,,,,,
Claude_Code,8,14,5,32,250,115,0,3,26,0,0,6
Copilot,121,126,67,458,1661,1993,12,44,301,5,15,167
Cursor,42,50,16,207,616,411,9,24,111,0,18,37
Devin,64,281,59,620,1904,1249,3,62,437,6,25,117
OpenAI_Codex,392,425,264,2570,10019,4338,7,207,1413,5,130,2029


In [37]:
# Convert 'created_at' to datetime objects and make them timezone-aware (UTC)
all_user_df['created_at'] = pd.to_datetime(all_user_df['created_at'], utc=True)

# Define a threshold for seniority (e.g., accounts created before a certain date) and make it timezone-aware (UTC)
seniority_threshold = pd.to_datetime('2024-01-01', utc=True) # Example threshold

# Categorize users
all_user_df['user_type'] = all_user_df['created_at'].apply(lambda x: 'Senior' if x < seniority_threshold else 'Newcomer')

# Display the counts of each user type
display(all_user_df['user_type'].value_counts())

,count
user_type,
Senior,55227
Newcomer,16962


In [39]:
# Merge all_pr_df and all_user_df to link pull requests with user types
merged_pr_user_df = pd.merge(all_pr_df, all_user_df[['id', 'user_type']], left_on='user_id', right_on='id', how='left', suffixes=('_pr', '_user'))

# Group by user type and agent and count the occurrences
user_type_agent_counts = merged_pr_user_df.groupby(['user_type', 'agent']).size().unstack(fill_value=0)

# Display the results
display(user_type_agent_counts)

agent,Claude_Code,Copilot,Cursor,Devin,OpenAI_Codex
user_type,,,,,
Newcomer,735,142,11389,29744,218443
Senior,4386,466,21527,0,595547


In [40]:
#assure that seniors never used Devin, by getting how many users used Devin , if the count of the users is same as the resutl, that means our anaylsis is correct

In [44]:
# Count the number of unique users who used the 'Devin' agent
devin_users_count = merged_pr_user_df[merged_pr_user_df['agent'] == 'Devin']['user_id'].nunique()

print(f"Number of unique users who used Devin: {devin_users_count}")

# Get the count of Devin users in the 'Newcomer' category from the user_type_agent_counts table
newcomer_devin_count = user_type_agent_counts.loc['Newcomer', 'Devin']

print(f"Number of Newcomer users who used Devin: {newcomer_devin_count}")

# Check if the counts are the same
if devin_users_count == newcomer_devin_count:
    print(f"The analysis is correct: Seniors did not use Devin.{devin_users_count}")
else:
    print(f"The analysis is incorrect: There are Devin users who are not Newcomers. {devin_users_count}")

Number of unique users who used Devin: 1
Number of Newcomer users who used Devin: 29744
The analysis is incorrect: There are Devin users who are not Newcomers. 1


In [ ]:
#why non of seniors used Devin ?

In [10]:
all_pr_df

,id,number,title,body,agent,user_id,user,state,created_at,closed_at,merged_at,repo_id,repo_url,html_url
0,3264016139,1688,`metta code` --> `metta clip` and additional p...,Remove unused `root_key` variable to fix ruff ...,Claude_Code,37011,jacklionheart,closed,2025-07-25T18:15:36Z,2025-07-25T19:17:23Z,2025-07-25T19:17:23Z,8.439884e+08,https://api.github.com/repos/Metta-AI/metta,https://github.com/Metta-AI/metta/pull/1688
1,3264021033,41,feat: Comprehensive ruff error resolution with...,## 🎯 Mission Accomplished: 100% Ruff Error Res...,Claude_Code,131842369,Draco3310,open,2025-07-25T18:17:57Z,None,None,9.920635e+08,https://api.github.com/repos/Draco3310/Gal-Fri...,https://github.com/Draco3310/Gal-Friday2/pull/41
2,3264042289,1600,Add Evals frontend implementation plan and HTM...,\nCreate comprehensive implementation plan for...,Claude_Code,6766889,justicart,closed,2025-07-25T18:26:15Z,2025-07-25T23:19:14Z,None,9.267118e+08,https://api.github.com/repos/bolt-foundry/bolt...,https://github.com/bolt-foundry/bolt-foundry/p...
3,3264042318,1601,Add 4 new BfDs components for Evals interface ...,\nPhase 1 component creation for the Evals fro...,Claude_Code,6766889,justicart,closed,2025-07-25T18:26:16Z,2025-07-25T23:19:11Z,None,9.267118e+08,https://api.github.com/repos/bolt-foundry/bolt...,https://github.com/bolt-foundry/bolt-foundry/p...
4,3264067496,3,🚀 Complete Frontend-Backend API Integration wi...,## 🎯 Summary\n\nThis PR completes the **fronte...,Claude_Code,42357482,twitchyvr,closed,2025-07-25T18:39:14Z,2025-07-25T18:48:47Z,2025-07-25T18:48:47Z,1.025871e+09,https://api.github.com/repos/twitchyvr/Spaghetti,https://github.com/twitchyvr/Spaghetti/pull/3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932786,2858495710,79,chore: update RPC endpoint and build target,# Update RPC Endpoint and Build Target\n\nUpda...,Devin,158243242,devin-ai-integration[bot],closed,2025-02-17T17:53:32Z,2025-02-17T17:54:19Z,2025-02-17T17:54:19Z,9.274854e+08,https://api.github.com/repos/xiayuhkust/TuraWAP,https://github.com/xiayuhkust/TuraWAP/pull/79
932787,2858498986,6,feat: enhance recruitment management system wi...,# Enhanced Recruitment Management System\n\nTh...,Devin,158243242,devin-ai-integration[bot],closed,2025-02-17T17:55:34Z,2025-02-18T02:19:26Z,2025-02-18T02:19:26Z,9.334548e+08,https://api.github.com/repos/crb12546/musical-...,https://github.com/crb12546/musical-memory/pull/6
932788,2858525953,80,fix: revert to PR #73 OpenAI config with updat...,# Revert to PR #73 with Updated RPC Endpoint\n...,Devin,158243242,devin-ai-integration[bot],closed,2025-02-17T18:10:27Z,2025-02-17T18:13:05Z,2025-02-17T18:13:05Z,9.274854e+08,https://api.github.com/repos/xiayuhkust/TuraWAP,https://github.com/xiayuhkust/TuraWAP/pull/80
932789,2858527610,400,Add Doxygen Documentation for Public APIs,# Add Doxygen Documentation for Public APIs\n\...,Devin,158243242,devin-ai-integration[bot],closed,2025-02-17T18:11:27Z,2025-02-22T01:15:27Z,None,1.195796e+08,https://api.github.com/repos/wolfSSL/wolfTPM,https://github.com/wolfSSL/wolfTPM/pull/400


In [11]:
all_repo_df

,id,url,license,full_name,language,forks,stars
0,987641962,https://api.github.com/repos/1010-dev/senjudev...,None,1010-dev/senjudev-site,TypeScript,1.0,0.0
1,990249393,https://api.github.com/repos/106-/HellSinkerWa...,None,106-/HellSinkerWallPaper,Java,0.0,0.0
2,1009549206,https://api.github.com/repos/1genadam/tileshop...,None,1genadam/tileshop-rag,Python,0.0,0.0
3,983546765,https://api.github.com/repos/1kimnet/ETL-pipeline,None,1kimnet/ETL-pipeline,Python,0.0,0.0
4,1024190983,https://api.github.com/repos/20m61/lightningta...,None,20m61/lightningtalk-circle,JavaScript,0.0,0.0
...,...,...,...,...,...,...,...
116206,1020305485,https://api.github.com/repos/zx8086/kong-konne...,None,zx8086/kong-konnect-app,Svelte,0.0,0.0
116207,976509925,https://api.github.com/repos/zx8086/mcp-server...,None,zx8086/mcp-server-couchbase,TypeScript,0.0,0.0
116208,989288917,https://api.github.com/repos/zx8086/mcp-server...,Apache-2.0,zx8086/mcp-server-elasticsearch,TypeScript,0.0,0.0
116209,915873255,https://api.github.com/repos/zxcpluto/First,None,zxcpluto/First,None,0.0,0.0


In [12]:
all_user_df

,id,login,followers,following,created_at
0,149159513.0,00012122Cs,0.0,0.0,2023-10-27T10:22:50Z
1,86906973.0,000Sean000,0.0,2.0,2021-07-04T07:05:40Z
2,36679210.0,000alen,46.0,106.0,2018-02-20T21:00:55Z
3,201261210.0,000qhrey,0.0,1.0,2025-02-28T16:53:21Z
4,22735204.0,00125495,0.0,1.0,2016-10-10T01:24:34Z
...,...,...,...,...,...
72184,128873850.0,zztime04,0.0,1.0,2023-03-25T01:50:00Z
72185,46803451.0,zzy1099207684,0.0,1.0,2019-01-18T02:51:48Z
72186,35818234.0,zzzpotato,2.0,0.0,2018-01-25T20:59:53Z
72187,55004427.0,zzzsaft,1.0,0.0,2019-09-06T21:50:29Z


## AIDev-pop (repositories that have more than 100 stars)

In [13]:
# Basic
pr_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pull_request.parquet")
repo_df = pd.read_parquet("hf://datasets/hao-li/AIDev/repository.parquet")
user_df = pd.read_parquet("hf://datasets/hao-li/AIDev/user.parquet")

# Comments and reviews
pr_comments_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pr_comments.parquet")
pr_reviews_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pr_reviews.parquet")
pr_review_comments_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pr_review_comments_v2.parquet")

# Commits
pr_commits_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pr_commits.parquet")
pr_commit_details_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pr_commit_details.parquet")

# Related issues
related_issue_df = pd.read_parquet("hf://datasets/hao-li/AIDev/related_issue.parquet")
issue_df = pd.read_parquet("hf://datasets/hao-li/AIDev/issue.parquet")

# Events
pr_timeline_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pr_timeline.parquet")

# Task type
pr_task_type_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pr_task_type.parquet")

# Human-PR
human_pr_df = pd.read_parquet("hf://datasets/hao-li/AIDev/human_pull_request.parquet")
human_pr_task_type_df = pd.read_parquet("hf://datasets/hao-li/AIDev/human_pr_task_type.parquet")

In [14]:
print(f"len(pr_df): {len(pr_df)}")
print(f"len(repo_df): {len(repo_df)}")
print(f"len(user_df): {len(user_df)}")

print(f"len(pr_comments_df): {len(pr_comments_df)}")
print(f"len(pr_reviews_df): {len(pr_reviews_df)}")
print(f"len(pr_review_comments_df): {len(pr_review_comments_df)}")

print(f"len(pr_commits_df): {len(pr_commits_df)}")
print(f"len(pr_commit_details_df): {len(pr_commit_details_df)}")


print(f"len(related_issue_df): {len(related_issue_df)}")
print(f"len(issue_df): {len(issue_df)}")

print(f"len(pr_timeline_df): {len(pr_timeline_df)}")

print(f"len(pr_task_type_df): {len(pr_task_type_df)}")

print(f"len(human_pr_df): {len(human_pr_df)}")
print(f"len(human_pr_task_type_df): {len(human_pr_task_type_df)}")

len(pr_df): 33596
len(repo_df): 2807
len(user_df): 1796
len(pr_comments_df): 39122
len(pr_reviews_df): 28875
len(pr_review_comments_df): 26868
len(pr_commits_df): 88576
len(pr_commit_details_df): 711923
len(related_issue_df): 4923
len(issue_df): 4614
len(pr_timeline_df): 325500
len(pr_task_type_df): 33596
len(human_pr_df): 6618
len(human_pr_task_type_df): 6618


### Extract patches from merged Agentic-PRs

In [15]:
merged_pr_mask = ~pr_df['merged_at'].isna()
merged_pr_ids = pr_df[merged_pr_mask]['id']
patch_mask = pr_commit_details_df['pr_id'].isin(merged_pr_ids)
pr_commit_details_df[patch_mask]['patch'].dropna()

,patch
0,"@@ -0,0 +1,38 @@\n+name: Compile llms.txt\n+\n..."
1,"@@ -0,0 +1,47 @@\n+import os\n+from pathlib im..."
3,"@@ -1,47 +1,48 @@\n import os\n from pathlib i..."
4,"@@ -3,45 +3,60 @@\n \n \n def compile_llms_txt..."
6,"@@ -1,29 +1,34 @@\n-import os\n from pathlib i..."
...,...
711885,"@@ -2,6 +2,7 @@\n \n Servers focused on static..."
711918,"@@ -0,0 +1,53 @@\n+# numbagg Development Guide..."
711919,"@@ -49,5 +49,5 @@ uv run pytest numbagg/test/t..."
711920,"@@ -339,7 +339,7 @@\n ..."


In [16]:
pr_commit_details_df['is_na'] = pr_commit_details_df['patch'].isna()
merged = pd.merge(pr_commit_details_df, all_pr_df[['id', 'agent']], left_on='pr_id', right_on='id')
merged.groupby('agent').apply(lambda x: x['is_na'].value_counts())

/tmp/ipython-input-2143530944.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged.groupby('agent').apply(lambda x: x['is_na'].value_counts())


is_na,False,True
agent,,
Claude_Code,20749,2547
Copilot,154744,32503
Cursor,35162,2676
Devin,152064,17403
OpenAI_Codex,279002,15073


### Check PR type

In [17]:
pr_task_type_df['type'].value_counts()

,count
type,
feat,14450
fix,8106
docs,3887
test,2356
refactor,2288
chore,896
build,627
ci,411
perf,340
